In [9]:
# some useful mysklearn package import statements and reloads
import importlib
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

# uncomment once you paste your myclassifiers.py into mysklearn package
import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

# get table and header
table, header = myutils.get_tables("input_data/webmd.csv")



In [10]:
drug_table = MyPyTable(header, table)
drug_table.remove_rows_with_missing_values()

labels = ["Drug", "Age", "Condition", "Date", "EaseofUse", "Satisfaction", "Sex", "UsefulCount", "Effectiveness"]

# get X and y for stratified k fold cross validation
X = []
y = []
for i in range(len(drug_table.data)):
    # create X, y, remove class label from X
    X.append(drug_table.data) 
    y.append(drug_table.data[i][-1]) # class label

# get folds
X_train_folds, X_test_folds = myevaluation.stratified_kfold_cross_validation(X, y, n_splits=5, random_state=None, shuffle=True)
# create training/testing sets out of the folds
X_train_sets_bl, y_train_sets_bl = myutils.get_sets_from_folds(X, y, X_train_folds)
X_test_sets_bl, y_test_sets_bl = myutils.get_sets_from_folds(X, y, X_test_folds)

k_neighbors = MyKNeighborsClassifier(n_neighbors=7)

myutils.fit_predict_classification(X_train_sets_bl, y_train_sets_bl, 
    X_test_sets_bl, y_test_sets_bl, k_neighbors, "K Neighbors", labels)
